## Data Wrangling - Zipcode Aggregation of All Datas
from FRED public data and Zillow public data

In [2]:
#Import pandas, matplotlib.pyplot, and seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

In [3]:
#change directory to get data
path= '/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim'
os.chdir(path)

In [4]:
os.listdir()

['.gitkeep',
 '.ipynb_checkpoints',
 'ACS_rentsPrices_2011_2018.csv',
 'ACS_zillow_2011_2018.csv',
 'Annual_fredData_2011_2019',
 'df3_1956',
 'df_homePrices_2011_2018',
 'df_homePrices_2011_2020',
 'df_homePrices_2011_2020.csv',
 'df_rentPrices_2014_2020',
 'df_rentPrices_2014_2020.csv',
 'VacancyRate_Zipcode_2011_2018',
 'VacancyRate_Zipcode_2011_2018.csv',
 'zillow_rentPrices_2014_2020.csv']

In [6]:
#load rental data
df_fred = pd.read_csv('Annual_fredData_2011_2019')
df_homes = pd.read_csv("df_homePrices_2011_2020.csv", converters={'RegionName': lambda x: str(x)})
df_rents = pd.read_csv('ACS_rentsPrices_2011_2018.csv', dtype={'RegionID': object, 'RegionName': object})
df_vacancy = pd.read_csv('VacancyRate_Zipcode_2011_2018.csv', dtype={'Zipcode': object})

#secondary rental data
ACS_Rents2011_18 = pd.read_csv('ACS_zillow_2011_2018.csv', dtype={'RegionID': object, 'RegionName': object})
zillow_Rents2014_20 = pd.read_csv('zillow_rentPrices_2014_2020.csv', dtype={'RegionID': object, 'RegionName': object})

### Data Definition

In [7]:
df_vacancy

,Zipcode,Vacancy_Rate%,MOE-VacancyRate%,Year
0,02333,3.024027,2.199925,2011
1,02338,3.116343,2.948791,2011
2,02339,4.464646,2.066438,2011
3,02341,3.586322,2.340722,2011
4,02343,3.732901,2.926524,2011
...,...,...,...,...
264955,98279,51.219512,10.993457,2018
264956,98280,51.329243,12.777549,2018
264957,98311,6.540162,1.960476,2018
264958,98326,28.537736,14.679524,2018


In [8]:
df_homes

,SizeRank,RegionName,State,City,Metro,CountyName,HomePrice,Year
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,784477.42,2011
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,400695.83,2011
2,2,10023,NY,New York,New York-Newark-Jersey City,New York County,821072.58,2011
3,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,265425.17,2011
4,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,507804.42,2011
...,...,...,...,...,...,...,...,...
303665,34430,43523,OH,Malinta,NaN,Henry County,84331.11,2020
303666,34430,7703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,579932.56,2020
303667,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,42896.56,2020
303668,35187,822,LA,Choudrant,Ruston,Lincoln Parish,195888.67,2020


### Data Cleaning 1 - HomePrices

In [9]:
#add leading zeros to zipcode in home dataframe
df_homes['RegionName'] = df_homes['RegionName'].apply(lambda x: '{0:0>5}'.format(x))

In [10]:
df_homes

,SizeRank,RegionName,State,City,Metro,CountyName,HomePrice,Year
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,784477.42,2011
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,400695.83,2011
2,2,10023,NY,New York,New York-Newark-Jersey City,New York County,821072.58,2011
3,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,265425.17,2011
4,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,507804.42,2011
...,...,...,...,...,...,...,...,...
303665,34430,43523,OH,Malinta,NaN,Henry County,84331.11,2020
303666,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,579932.56,2020
303667,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,42896.56,2020
303668,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,195888.67,2020


In [11]:
#rename Zipcode column
df_homes = df_homes.rename(columns = {'RegionName':'Zipcode'})
#check for NaNs
df_homes.isna().sum()

SizeRank          0
Zipcode           0
State             0
City              0
Metro         71180
CountyName        0
HomePrice     11066
Year              0
dtype: int64

In [12]:
#subset df_homes data for year 2011-2018 (for merging with FRED data)
df_homes_2011_2018 = df_homes.loc[df_homes['Year'] < 2019]

In [13]:
#subset df_homes data for year 2014-2020 (for merging with rent data)
#df_homes_2014_2020 = df_homes.loc[df_homes['Year'] > 2013]

In [25]:
#df_homes_2014_2020[df_homes_2014_2020.Year == 2014]

,SizeRank,Zipcode,State,City,Metro,CountyName,HomePrice,Year
91101,0,10025,NY,New York,New York-Newark-Jersey City,New York County,968761.75,2014
91102,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75,2014
91103,2,10023,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,2014
91104,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00,2014
91105,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92,2014
...,...,...,...,...,...,...,...,...
121463,34430,43523,OH,Malinta,NaN,Henry County,67959.33,2014
121464,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,NaN,2014
121465,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,NaN,2014
121466,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,140783.33,2014


In [41]:
df_homes_2011_2018

,SizeRank,Zipcode,State,City,Metro,CountyName,HomePrice,Year
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,784477.42,2011
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,400695.83,2011
2,2,10023,NY,New York,New York-Newark-Jersey City,New York County,821072.58,2011
3,3,77494,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,265425.17,2011
4,4,60614,IL,Chicago,Chicago-Naperville-Elgin,Cook County,507804.42,2011
...,...,...,...,...,...,...,...,...
242931,34430,43523,OH,Malinta,NaN,Henry County,81699.58,2018
242932,34430,07703,NJ,Eatontown,New York-Newark-Jersey City,Monmouth County,543333.30,2018
242933,34430,47865,IN,Carlisle,Terre Haute,Sullivan County,40215.62,2018
242934,35187,00822,LA,Choudrant,Ruston,Lincoln Parish,178884.17,2018


In [27]:
#check %NaNs in home_price data
df_homes_2011_2018.isna().sum()/len(df_homes_2011_2018)*100

SizeRank       0.000000
Zipcode        0.000000
State          0.000000
City           0.000000
Metro         23.439918
CountyName     0.000000
HomePrice      4.555109
Year           0.000000
dtype: float64

In [ ]:
#replace home price NaNs with first:
    #diff = (price of year you have) - (mean of a year you have) 
    #then replace NaN with 'diff' + (mean of NaN year)

### Data Cleaning 2 - RentPrices

In [40]:
df_rents

,Zipcode,RentPrice,Year
0,02333,1368.536,2011
1,02338,1311.076,2011
2,02339,1484.626,2011
3,02341,1266.816,2011
4,02343,1524.006,2011
...,...,...,...
264955,98279,1059.870,2018
264956,98280,993.850,2018
264957,98311,1533.500,2018
264958,98326,778.990,2018


In [19]:
#add leading zeros to zipcode in rent dataframe
df_rents['RegionName'] = df_rents['RegionName'].apply(lambda x: '{0:0>5}'.format(x))

In [20]:
df_rents

,RegionName,RentPrice,Year
0,02333,1368.536,2011
1,02338,1311.076,2011
2,02339,1484.626,2011
3,02341,1266.816,2011
4,02343,1524.006,2011
...,...,...,...
264955,98279,1059.870,2018
264956,98280,993.850,2018
264957,98311,1533.500,2018
264958,98326,778.990,2018


In [21]:
#rename Zipcode column
df_rents = df_rents.rename(columns={'RegionName': 'Zipcode'})

In [28]:
#check for % NaNs
df_rents.isna().sum()/len(df_rents)*100

Zipcode      0.00000
RentPrice    7.51321
Year         0.00000
dtype: float64

In [29]:
#replace home price NaNs with first:
    #diff = (price of year you have) - (mean of a year you have) 
    #then replace NaN with 'diff' + (mean of NaN year)

### Data Cleaning 3 - FRED Economic Data

In [30]:
df_fred.head()

,DATE,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,2011-01-01,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,2012-01-01,0.750000,56912.0,0.727269,8.075000,783.750000,278995.583333
2,2013-01-01,0.750000,58904.0,0.686773,7.358333,928.166667,335207.333333
3,2014-01-01,0.750000,58001.0,0.727518,6.158333,1000.250000,382868.333333
4,2015-01-01,0.770833,60987.0,0.730641,5.275000,1106.750000,438118.333333


In [31]:
#change year to match df_vacancy format
df_fred.DATE = df_fred.DATE.str.replace('-01-01','')

In [32]:
#change DATE column name to Year
df_fred = df_fred.rename(columns = {'DATE':'Year'})

In [33]:
#subset FRED data to 2011-2018 to prep for merge
df_fred_2011_2018 = df_fred.iloc[0:8]
df_fred_2011_2018 = df_fred_2011_2018.astype({'Year': 'int32'})

In [34]:
df_fred_2011_2018.dtypes

Year                       int32
int_rate                 float64
med_hIncome              float64
uspop_growth             float64
unemplt_rate             float64
newHouse_starts          float64
resConstruct_spending    float64
dtype: object

In [36]:
df_vacancy.dtypes

Zipcode              object
Vacancy_Rate%       float64
MOE-VacancyRate%    float64
Year                  int64
dtype: object

In [38]:
df_fred_2011_2018

,Year,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,2012,0.750000,56912.0,0.727269,8.075000,783.750000,278995.583333
2,2013,0.750000,58904.0,0.686773,7.358333,928.166667,335207.333333
3,2014,0.750000,58001.0,0.727518,6.158333,1000.250000,382868.333333
4,2015,0.770833,60987.0,0.730641,5.275000,1106.750000,438118.333333
5,2016,1.020833,62898.0,0.716669,4.875000,1176.583333,485822.500000
6,2017,1.625000,63761.0,0.631008,4.341667,1207.416667,546020.166667
7,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


### Data Joining

In [46]:
#merge rent prices and home prices (2011-2018)
df_rent_merged_homes = pd.merge(df_rents, df_homes_2011_2018,how='left', on=['Year', 'Zipcode'])
df_rent_merged_homes

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice
0,02333,1368.536,2011,8782.0,MA,East Bridgewater,Boston-Cambridge-Newton,Plymouth County,NaN
1,02338,1311.076,2011,11179.0,MA,Halifax,Boston-Cambridge-Newton,Plymouth County,274920.17
2,02339,1484.626,2011,8621.0,MA,Hanover,Boston-Cambridge-Newton,Plymouth County,415097.50
3,02341,1266.816,2011,10079.0,MA,Hanson,Boston-Cambridge-Newton,Plymouth County,NaN
4,02343,1524.006,2011,9640.0,MA,Holbrook,Boston-Cambridge-Newton,Norfolk County,247510.42
...,...,...,...,...,...,...,...,...,...
264955,98279,1059.870,2018,23400.0,WA,Olga,NaN,San Juan County,552805.42
264956,98280,993.850,2018,25265.0,WA,Eastsound,NaN,San Juan County,678499.00
264957,98311,1533.500,2018,4981.0,WA,Bremerton,Bremerton-Silverdale,Kitsap County,314320.83
264958,98326,778.990,2018,26185.0,WA,Clallam Bay,Port Angeles,Clallam County,150193.17


In [47]:
#merge df_vacancy + FRED data 2011-2018
df_vacany_merged_fred = pd.merge(df_vacancy, df_fred_2011_2018, how='left', on='Year')
df_vacany_merged_fred

,Zipcode,Vacancy_Rate%,MOE-VacancyRate%,Year,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,02333,3.024027,2.199925,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,02338,3.116343,2.948791,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
2,02339,4.464646,2.066438,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
3,02341,3.586322,2.340722,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
4,02343,3.732901,2.926524,2011,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
...,...,...,...,...,...,...,...,...,...,...
264955,98279,51.219512,10.993457,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264956,98280,51.329243,12.777549,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264957,98311,6.540162,1.960476,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264958,98326,28.537736,14.679524,2018,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


In [49]:
#merge all four datasets together
df_complete = pd.merge(df_rent_merged_homes, df_vacany_merged_fred, how='left', on=['Year', 'Zipcode'])
df_complete

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,02333,1368.536,2011,8782.0,MA,East Bridgewater,Boston-Cambridge-Newton,Plymouth County,NaN,3.024027,2.199925,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,02338,1311.076,2011,11179.0,MA,Halifax,Boston-Cambridge-Newton,Plymouth County,274920.17,3.116343,2.948791,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
2,02339,1484.626,2011,8621.0,MA,Hanover,Boston-Cambridge-Newton,Plymouth County,415097.50,4.464646,2.066438,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
3,02341,1266.816,2011,10079.0,MA,Hanson,Boston-Cambridge-Newton,Plymouth County,NaN,3.586322,2.340722,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
4,02343,1524.006,2011,9640.0,MA,Holbrook,Boston-Cambridge-Newton,Norfolk County,247510.42,3.732901,2.926524,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264955,98279,1059.870,2018,23400.0,WA,Olga,NaN,San Juan County,552805.42,51.219512,10.993457,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264956,98280,993.850,2018,25265.0,WA,Eastsound,NaN,San Juan County,678499.00,51.329243,12.777549,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264957,98311,1533.500,2018,4981.0,WA,Bremerton,Bremerton-Silverdale,Kitsap County,314320.83,6.540162,1.960476,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264958,98326,778.990,2018,26185.0,WA,Clallam Bay,Port Angeles,Clallam County,150193.17,28.537736,14.679524,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


In [50]:
#check NaN numbers
df_complete.isna().sum()

Zipcode                      0
RentPrice                19907
Year                         0
SizeRank                 27056
State                    27056
City                     27056
Metro                    83008
CountyName               27056
HomePrice                37378
Vacancy_Rate%                0
MOE-VacancyRate%             0
int_rate                     0
med_hIncome                  0
uspop_growth                 0
unemplt_rate                 0
newHouse_starts              0
resConstruct_spending        0
dtype: int64

In [51]:
#check NaN as a percentage
df_complete.isna().sum()/len(df_complete)*100

Zipcode                   0.000000
RentPrice                 7.513210
Year                      0.000000
SizeRank                 10.211353
State                    10.211353
City                     10.211353
Metro                    31.328502
CountyName               10.211353
HomePrice                14.107035
Vacancy_Rate%             0.000000
MOE-VacancyRate%          0.000000
int_rate                  0.000000
med_hIncome               0.000000
uspop_growth              0.000000
unemplt_rate              0.000000
newHouse_starts           0.000000
resConstruct_spending     0.000000
dtype: float64

In [53]:
df_complete.describe()

,RentPrice,Year,SizeRank,HomePrice,Vacancy_Rate%,MOE-VacancyRate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
count,245053.000000,264960.000000,237904.000000,2.275820e+05,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000,264960.000000
mean,1067.691442,2014.500000,15646.706806,1.846688e+05,17.675088,7.996235,1.109375,60351.000000,0.682779,6.113542,1007.885417,410836.197918
std,491.626977,2.291292,9424.124602,1.858222e+05,16.437987,10.221583,0.583590,2846.855913,0.068232,1.719867,208.944873,109740.019064
min,19.960000,2011.000000,0.000000,1.042183e+04,0.000000,0.000000,0.750000,56912.000000,0.522337,3.891667,611.916667,255208.583333
25%,781.625000,2012.750000,7531.000000,8.735608e+04,6.977963,2.491948,0.750000,57756.000000,0.672832,4.741667,892.062500,321154.395833
50%,942.446000,2014.500000,15164.000000,1.340185e+05,12.799977,4.855487,0.760417,59945.500000,0.718344,5.716667,1053.500000,410493.333333
75%,1204.816000,2016.250000,23514.000000,2.141746e+05,22.580645,9.359872,1.171875,63113.750000,0.727331,7.537500,1184.291667,500871.916667
max,5620.320000,2018.000000,34430.000000,6.141946e+06,100.000000,100.000000,2.458333,64324.000000,0.730641,8.933333,1248.250000,564448.750000


### Save Data

In [52]:
df_complete.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim/master_complete_for_EDA.csv', index=False)

### Notes


In [ ]:
#need to save dataframe for 2014-2020 data (for predictions?)
#NaNs
    #find table that has state, city, metroarea, county name by zipcode to deal with thses NaNs
    #don't worry about SizeRank..??
    #rent price and home price try below... 
    #replace home price NaNs with first:
        #diff = (price of year you have) - (mean of a year you have) 
        #then with NaN do 'diff' + (mean of NaN year)

### Summary